In [46]:
from lusidtools.jupyter_tools import toggle_code

"""Compliance in LUSID

Attributes
----------
Compliance
OMS
Pre/Post trade checks
"""

toggle_code("Toggle Docstring")

## Configuring and Running Pre-Trade Compliance
In this example we demonstrate the configuration of several simple pre-trade rules, run them against a sample portfolio. Then we will raise some orders on that portfolio and re-check pre-trade compliance. Lastly, we will allocate partially against the newly created order and run a post-trade compliance. 

## Imports

In [3]:
import lusid
import lusid.models as models
import lusid.api as la
import lusid.models as lm
from lusid.models.upsert_compliance_rule_request import UpsertComplianceRuleRequest
from lusid.models.reference_list_request import ReferenceListRequest
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os
import datetime
from datetime import datetime, timedelta, time, date
import pytz

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.12555.0


In [4]:
# define some APIs
properties_api = api_factory.build(la.PropertyDefinitionsApi)
referencelist_api = api_factory.build(la.ReferenceListsApi)
compliance_api = api_factory.build(la.ComplianceApi)
aggregation_api = api_factory.build(la.AggregationApi)
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
portfolio_groups_api = api_factory.build(la.PortfolioGroupsApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
instruments_api = api_factory.build(la.InstrumentsApi)
allocations_api = api_factory.build(lusid.api.AllocationsApi)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)

In [5]:
# Specify a unique scope and code to segregate data in this tutorial from others
scope = "simplecompliance2"
portfolio_code='EQUITY_UK'
portfolio_group_code='EQUITY_UK_GROUP'
print(f"'{scope}\{portfolio_code}' scope and code created.")

'simplecompliance2\EQUITY_UK' scope and code created.


## 1. Create instruments, portfolio and transactions to work with


### 1.0 Load transaction data

In [6]:
df = pd.read_csv("data/equity_transactions.csv")
df
#df.drop(df[df['instrument_type'] == 'cash'].index, inplace=True)


portfolio_code                       portfolio_name  \
0       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
1       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
2       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
3       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
4       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
5       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
6       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
7       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
8       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
9       EQUITY_UK  LUSID's top 10 FTSE stock portfolio   
10      EQUITY_UK  LUSID's top 10 FTSE stock portfolio   

   portfolio_base_currency          ISIN    sedol instrument_type  \
0                      GBP  GB0002162385  0216238          equity   
1                      GBP  GB00BH0P3Z91  BH0P3Z9          equity   
2                      GBP  GB0031348658  3134865          equity   
3                      GBP  GB0007980591  0798059          equity   
4                      GBP  GB0005405286  0540528          equity   
5                      GBP  GB0006043169  0604316          equity   
6                      GBP  GB0008847096  0884709          equity   
7                      GBP  GB00BGDT3G23  BGDT3G2          equity   
8                      GBP  GB00BH4HKS39  BH4HKS3          equity   
9                      GBP  GB00B1XZS820  B1XZS82          equity   
10                     GBP  ES0177542018  B5M6XQ7          equity   

   instrument_id                name              sector  txn_id txn_type  \
0        EQ_1234               Aviva  Financial Services   txn-1  StockIn   
1        EQ_1235                 BHP              Mining   txn-2  StockIn   
2        EQ_1236            Barclays  Financial Services   txn-3  StockIn   
3        EQ_1237                  BP         Oil and Gas   txn-4  StockIn   
4        EQ_1238                HSBC  Financial Services   txn-5  StockIn   
5        EQ_1239           Morrisons      Food and Drink   txn-6  StockIn   
6        EQ_1240               Tesco      Food and Drink   txn-7  StockIn   
7        EQ_1241           Rightmove         Real Estate   txn-8  StockIn   
8        EQ_1242            vodafone  Telecommunications   txn-9  StockIn   
9        EQ_1243  Anglo American plc              Mining  txn-10  StockIn   
10       EQ_1244       IAG group plc              travel  txn-11  StockIn   

   txn_trade_date txn_settle_date  txn_units  txn_price  txn_consideration  \
0      02/01/2020      04/01/2020     120000       4.23             600000   
1      02/01/2020      04/01/2020      60000      17.89            1080000   
2      02/01/2020      04/01/2020     150000       1.80             300000   
3      02/01/2020      04/01/2020     100000       4.75             500000   
4      02/01/2020      04/01/2020      20000       5.89             120000   
5      15/01/2020      17/01/2020     180000       1.87             360000   
6      16/01/2020      18/01/2020       4000       2.47              36000   
7      15/01/2020      17/01/2020      80000       6.59             480000   
8      15/01/2020      17/01/2020     450000       1.56             450000   
9      15/01/2020      17/01/2020      35000      21.68             700000   
10     15/01/2020      17/01/2020       1000       1.49               1490   

   currency      strategy  cash_transactions  
0       GBP  ftse_tracker                NaN  
1       GBP  ftse_tracker                NaN  
2       GBP  ftse_tracker                NaN  
3       GBP  ftse_tracker                NaN  
4       GBP  ftse_tracker                NaN  
5       GBP  ftse_tracker                NaN  
6       GBP  ftse_tracker                NaN  
7       GBP  ftse_tracker                NaN  
8       GBP  ftse_tracker                NaN  
9       GBP  ftse_tracker                NaN  
10      GBP  ftse_tracker                NaN

### 1.1 Load instruments

In [7]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ISIN",
        "Sedol": "sedol",
    },
    "required": {"name": "name"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
    property_columns=["sector"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

success  failed  errors
0       11       0       0

### 1.2 Create portfolio

In [8]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$USD",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

success  failed  errors
0        1       0       0

### 1.3 Create portfolio group

In [9]:
def create_portfolio_group(portfolio_groups_api, scope, code, portfolios):

    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=5000)

    try:
        portfolio_groups_api.delete_portfolio_group(
            scope=scope,
            code=code)
    except:
        pass
    
    group_request = lm.CreatePortfolioGroupRequest(
        code=code,
        display_name=code,
        values=portfolios,
        sub_groups=None,
        description=None,
        created=portfolio_creation_date)

    portfolio_group = portfolio_groups_api.create_portfolio_group(
        scope=scope,
        create_portfolio_group_request=group_request)
    
    return portfolio_group

In [10]:
result = create_portfolio_group(portfolio_groups_api, scope, portfolio_group_code, [lm.ResourceId(scope=scope, code=portfolio_code)])


### 1.4 Load transactions

In [11]:
transaction_mapping = {
    "identifier_mapping": {"ClientInternal": "instrument_id",},
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

success  failed  errors
0        1       0       0

### 1.5 Load quotes

In [12]:
quotes_df = pd.read_csv("data/equity_quotes.csv")
# Compliance runs at latest asat
quotes_df['quote_date'] = quotes_df['quote_date'].apply(lambda s: datetime.now(pytz.UTC).strftime("%d-%b-%y"))
quotes_df

ISIN quote_date    bid    mid    ask
0   GB0002162385  07-Feb-24   2.29   2.30   2.31
1   GB00BH0P3Z91  07-Feb-24  12.81  12.87  12.93
2   GB0031348658  07-Feb-24   0.88   1.88   0.88
3   GB0007980591  07-Feb-24   3.06   3.08   3.10
4   GB0005405286  07-Feb-24   4.00   4.02   4.04
5   GB0006043169  07-Feb-24   1.87   1.88   1.89
6   GB0008847096  07-Feb-24   2.35   2.36   2.37
7   GB00BGDT3G23  07-Feb-24   4.63   4.65   4.67
8   GB00BH4HKS39  07-Feb-24   1.08   1.09   1.10
9   GB00B1XZS820  07-Feb-24  13.90  13.97  14.04
10  GB0000566504  07-Feb-24  15.45  15.97  15.04
11  GB0000566504  07-Feb-24  15.45  15.97  15.04
12  ES0177542018  07-Feb-24   1.49   1.50   1.51

In [13]:
price_fields = ["bid", "mid", "ask"]

quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Isin",
    "quote_id.effective_at": "quote_date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "ISIN",
    "metric_value.unit": "$USD",
}

for price_field in price_fields:

    quotes_mapping["quote_id.quote_series_id.field"] = f"${price_field}"
    quotes_mapping["metric_value.value"] = price_field

    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=quotes_df,
        mapping_required=quotes_mapping,
        mapping_optional={},
        file_type="quotes",
    )

    succ, failed, errors = format_quotes_response(result)
    display(
        pd.DataFrame(
            data=[
                {
                    f"[{price_field}] success": len(succ),
                    "failed": len(failed),
                    "errors": len(errors),
                }
            ]
        )
    )

[bid] success  failed  errors
0             12       0       0

[mid] success  failed  errors
0             12       0       0

[ask] success  failed  errors
0             12       0       0

### 1.6 Create recipes

In [14]:
for price_field in ["mid", "bid", "ask"]:

    # Create a recipe to perform a valuation
    configuration_recipe = models.ConfigurationRecipe(
        scope="User",
        code="valuation_recipe" + "_" + price_field,
        market=models.MarketContext(
            market_rules=[
                # define how to resolve the quotes
                models.MarketDataKeyRule(
                    key="Quote.Isin.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field=price_field,
                ),
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="Isin",
                default_scope=scope,
            ),
        ),
        pricing=models.PricingContext(
            options={"AllowPartiallySuccessfulEvaluation": True},
        ),
    )

    upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )


### 1.7 Run a test valuation

In [15]:
def generate_valuation_request(valuation_effectiveAt, price_field):

    # Create the valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(
            scope="User", code="valuation_recipe" + "_" + price_field
        ),
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Proportion"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
            models.AggregateSpec("Aggregation/Errors", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        filters=[models.PropertyFilter('Instrument/default/Name', 'NotEquals', 'GBP'),
                models.PropertyFilter('Instrument/default/Name', 'NotEquals', 'GBP Cash')
        ],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

In [16]:
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        datetime.now(pytz.UTC), "mid"
    )
)
pd.DataFrame(aggregation.data)



Instrument/default/Name  Proportion(Valuation/PvInReportCcy)  \
0                    Aviva                             0.080716   
1                      BHP                             0.225830   
2                 Barclays                             0.082471   
3                       BP                             0.090075   
4                     HSBC                             0.023513   
5                Morrisons                             0.098965   
6                Rightmove                             0.108791   
7                 vodafone                             0.143447   
8       Anglo American plc                             0.142993   
9            IAG group plc                             0.000439   
10                   Tesco                             0.002761   

    Sum(Valuation/PvInReportCcy)  Sum(Holding/default/Units)  \
0                       276000.0                    120000.0   
1                       772200.0                     60000.0   
2                       282000.0                    150000.0   
3                       308000.0                    100000.0   
4                        80400.0                     20000.0   
5                       338400.0                    180000.0   
6                       372000.0                     80000.0   
7                       490500.0                    450000.0   
8                       488950.0                     35000.0   
9                         1500.0                      1000.0   
10                        9440.0                      4000.0   

   Aggregation/Errors  
0                  []  
1                  []  
2                  []  
3                  []  
4                  []  
5                  []  
6                  []  
7                  []  
8                  []  
9                  []  
10                 []

## 2. Define properties and supporting data

In [17]:
def create_property_definition(properties_api, domain, scope, code, data_type):
    properties_api.create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain=domain,
            scope=scope,
            code=code,
            display_name=code,
            life_time="Perpetual",
            value_required=False,
            data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type)
        )
    )

In [18]:

try:
    create_property_definition(properties_api, "Portfolio", scope, 'Placeholder', "string")
except:
    pass

### 2.1 Decorate portfolio with properties

In [19]:
response = portfolios_api.upsert_portfolio_properties(
        scope=scope,
        code=portfolio_code,
        request_body={
            "Portfolio/{}/Placeholder".format(
                scope
            ): models.ModelProperty(
                key="Portfolio/{}/Placeholder".format(scope),
                value=models.PropertyValue(label_value='PlaceholderValue'),
            )
        },
    )

### 2.2 Create reference lists

We need a couple of empty lists to allow pass-through behaviour.

In [20]:
request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='empty-list'
    ),
    name="empty string list",
    description="some description",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=[]
    )
)

empty_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='empty-portfolio-list'
    ),
    name="empty portfolioid list",
    description="some description",
    tags=[],
    reference_list=lm.PortfolioIdList(
        reference_list_type='PortfolioIdList',
        values=[]
    )
)

empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='restricted-sectors'
    ),
    name="restricted sectors list",
    description="some description",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=["Mining"]
    )
)

empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)


## 3. Setup compliance rules

### 3.1 A simple PV contribution threshold check
Checking whether any holding is over 10% of portfolio PV

In [21]:
pvInReportCcy = lm.AddressKeyComplianceParameter("passed_validation", compliance_parameter_type='AddressKeyComplianceParameter')
pvInReportCcy._value = "Valuation/PvInReportCcy"

upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-1'),
    name='threshold-pv-check',
    description='Simple threshold PV check',
    template_id=lm.ResourceId(scope='system', code='PercentCheck'),
    variation='standard',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "Metric": pvInReportCcy,
        "UpperBound": lm.DecimalComplianceParameter(value="10",compliance_parameter_type='DecimalComplianceParameter'),
        "LowerBound": lm.DecimalComplianceParameter(value="-1",compliance_parameter_type='DecimalComplianceParameter'),
        "UpperWarning": lm.DecimalComplianceParameter(value="30",compliance_parameter_type='DecimalComplianceParameter'),
        "LowerWarning": lm.DecimalComplianceParameter(value="-1",compliance_parameter_type='DecimalComplianceParameter'),
        "FirstFilterPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Portfolio/{scope}/Placeholder",compliance_parameter_type='PropertyKeyComplianceParameter'),
        "FirstFilterPermittedValuesList": lm.StringListComplianceParameter(value=lm.ResourceId(scope="simplecompliance2", code="empty-list"),compliance_parameter_type='StringListComplianceParameter'),
        "SecondFilterPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Portfolio/{scope}/Placeholder",compliance_parameter_type='PropertyKeyComplianceParameter'),
        "SecondFilterPermittedValuesList": lm.StringListComplianceParameter(value=lm.ResourceId(scope="simplecompliance2", code="empty-list"),compliance_parameter_type='StringListComplianceParameter'),
        "GroupingPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/default/Isin",compliance_parameter_type='PropertyKeyComplianceParameter'),
        "Excludes": lm.PortfolioIdListComplianceParameter(value=lm.ResourceId(scope="simplecompliance2",code="empty-portfolio-list"),compliance_parameter_type='PortfolioIdListComplianceParameter')
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

### 3.2 A restricted sector check
Restricting mining sector in this rule

In [22]:
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-2'),
    name="restricted-sector-check",
    description='Simple sector restriction',
    template_id=lm.ResourceId(scope='system', code='PropertyValueNotInList'),
    variation='standard-not-in-list',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "PropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/simplecompliance2/sector", compliance_parameter_type='PropertyKeyComplianceParameter'),
        "ExclusivePropertyList": lm.StringListComplianceParameter(value=lm.ResourceId(scope="simplecompliance2", code="restricted-sectors"),compliance_parameter_type='StringListComplianceParameter'),
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

## 4. Run Compliance Rules

Currently the API enables us to run compliance as at now, rather than for a specific date.

In [26]:
run_response = compliance_api.run_compliance(run_scope=scope,rule_scope=scope,is_pre_trade=True,recipe_id_scope='User',recipe_id_code='valuation_recipe_mid')
run_code = run_response.run_id.code

print(f"Compliance run {run_response.run_id.scope}/{run_response.run_id.code} completed.")
print(f"Instigated: {run_response.instigated_at}, completed: {run_response.completed_at}")

Compliance run simplecompliance2/0b7f8f39-f15b-426b-b1d9-e3d8d8a5aa6f completed.
Instigated: 2024-02-07 10:00:16.475368+00:00, completed: 2024-02-07 10:00:18.527309+00:00


## 5. Analyse the compliance result¶
### 5.1 Identify rule-level failures
At a very coarse level we can determine which rules have been breached in this compliance run. We can also see a list of orders and portfolios affected by the breach.

The list of orders might well be wider than initially expected. This is because LUSID compliance makes no assumptions about which orders take precedence; it takes a holistic, contingent view about compliance taking into account all orders and existing positions.

Here you can see that rule-1 has failed, and one portfolio is affected by the failure. We'll drilldown into this rule result in the following section.

In [27]:
def rule_level_dataframe(run_summary):
    # Use the first result as a way of generating overall headers
    h = ['', '', '', '', '']
    c = ['Rule', 'Rule Description', 'Status', 'Affected Orders', 'Affected Portfolios']

    df = pd.DataFrame([c], columns=h)

    new_labels = pd.MultiIndex.from_arrays([df.columns, df.iloc[0]], names=['', ''])
    df = df.set_axis(new_labels, axis=1).iloc[1:]

    # Now build a row per result
    for d in run_summary.details:
        r = [f"{d.rule_id.scope}/{d.rule_id.code}", d.rule_description, d.status, len(d.affected_orders), len(d.affected_portfolios_details)]

        df.loc[len(df)] = r

    return df

In [28]:
run_summary = compliance_api.get_decorated_compliance_run_summary(scope=scope, code=run_code)

df = rule_level_dataframe(run_summary)

print(f"Rule-level results for run {run_summary.run_id.scope}/{run_summary.run_id.code}.")
display(df)

Rule-level results for run simplecompliance2/0b7f8f39-f15b-426b-b1d9-e3d8d8a5aa6f.


\
                       Rule           Rule Description  Status   
0  simplecompliance2/rule-2  Simple sector restriction  Failed   
1  simplecompliance2/rule-1  Simple threshold PV check  Failed   

                                       
  Affected Orders Affected Portfolios  
0               0                   1  
1               0                   1

### 5.2 Drilldown into specific rule results

#### 5.2.1 Simple PV threshold breach

`rule-1` failed the compliance check overall. This rule is a simple 10% PV contribution threshold check often used to control portfolio diversification, so a failure indicates that one or more instruments contributed more than 10%. Taking a look at the drilldown data for this rule we can see that `GB00B1XZS820`, `GB00BH4HKS39`, `GB00BGDT3G23` and `GB00BH0P3Z91` in the `EQUITY_UK` portfolio each contribute more than 10%.

The rule is structured to
- start with all holdings in portfolio group `EQUITY_GROUP_UK` (the initial group)
- filter out contributions *for portfolios in an excluded portfolios list*
- group contributions *by portfolio id*
- two filter steps, here configured not to exclude any contributions
- further group contributions for each portfolio *by Isin*
- finally, compare a pair of Results Used (Valuation/PVInReportCcy compared to portfolio-level Valuation/PVInReportCcy) *(checking that one is less than 10% of the other)*

This information is represented by the Lineage for each drilldown row; the Lineage can be used to get a high-level view of what's caused a rule breach, and a pointer to where to start more detailed investigations if needed.

In [29]:
rule_result1 = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code, rule_scope=scope, rule_code='rule-1')


filtered_breakdowns1 = [breakdown for breakdown in rule_result1.rule_result.rule_breakdown if getattr(breakdown, 'group_status', '') == 'Failed']
rule_result1_df=lusid_response_to_data_frame(filtered_breakdowns1)
display(rule_result1_df[['group_status','results_used.Valuation/PvInReportCcy','results_used.Portfolios.Valuation/PvInReportCcy',
                       'lineage.1.label','lineage.2.sub_label','lineage.5.sub_label','lineage.5.information']])


group_status  results_used.Valuation/PvInReportCcy  \
0       Failed                              772200.0   
1       Failed                              372000.0   
2       Failed                              490500.0   
3       Failed                              488950.0   

   results_used.Portfolios.Valuation/PvInReportCcy            lineage.1.label  \
0                                        3419390.0  WithoutExcludedPortfolios   
1                                        3419390.0  WithoutExcludedPortfolios   
2                                        3419390.0  WithoutExcludedPortfolios   
3                                        3419390.0  WithoutExcludedPortfolios   

           lineage.2.sub_label lineage.5.sub_label    lineage.5.information  
0  simplecompliance2/EQUITY_UK        GB00BH0P3Z91  Instrument/default/Isin  
1  simplecompliance2/EQUITY_UK        GB00BGDT3G23  Instrument/default/Isin  
2  simplecompliance2/EQUITY_UK        GB00BH4HKS39  Instrument/default/Isin  
3  simplecompliance2/EQUITY_UK        GB00B1XZS820  Instrument/default/Isin

### 5.2.2 Restricted sector breach
Rule-2 failed the compliance check overall. This rule prevents trade in a restricted set of sectors (specifically just Mining in this case), so a failure indicates that one or more holdings across the whole portfolio group are of an instrument from a restricted sector.

In [30]:
rule_result2 = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code, rule_scope=scope, rule_code='rule-2')

filtered_breakdowns2 = [breakdown for breakdown in rule_result2.rule_result.rule_breakdown if getattr(breakdown, 'group_status', '') == 'Failed']

rule_result2_df=lusid_response_to_data_frame(filtered_breakdowns2)
display(rule_result2_df[['group_status','lineage.1.sub_label']])

group_status                         lineage.1.sub_label
0       Failed  Instrument/simplecompliance2/sector=Mining

## 6 Upsert an order to show its effect
We will now upsert an order and run compliance rule-1 against that. It should flag as i have an order that breaches the PV threshold.

In [31]:
orders_df = pd.read_csv('data/live_orders.csv')
orders_df

portfolio instrument_name client_internal          isin          figi  \
0  EQUITY_UK   IAG group plc         EQ_1244  ES0177542018  BBG00J9WBYB9   
1  EQUITY_UK   IAG group plc         EQ_1244  ES0177542018  BBG00J9WBYB9   
2  EQUITY_UK   IAG group plc         EQ_1244  ES0177542018  BBG00J9WBYB9   
3  EQUITY_UK   IAG group plc         EQ_1244  ES0177542018  BBG00J9WBYB9   

   quantity  price currency order_id side    type state  limit_price  \
0  10000000   1.48      GBP   ORD005  Buy   Limit   New         1.48   
1      1000   1.48      GBP   ORD006  Buy   Limit   New         1.49   
2      3000    NaN      GBP   ORD007  Buy    Stop   New          NaN   
3      4000    NaN      GBP   ORD008  Buy  Market   New          NaN   

  limit_currency  stop_price stop_currency  
0            GBP         NaN           NaN  
1            GBP         NaN           NaN  
2            NaN         1.5           GBP  
3            NaN         NaN           NaN

In [32]:
#generating the order via order API
order_requests = defaultdict(list)
order_sets = defaultdict(list)
responses = []

for index, order in orders_df.iterrows():
    
    request = lm.OrderRequest(
            id=lm.ResourceId(
                scope=scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': order['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=scope,
                code=portfolio_code
            ),
            state=order['state'],
            type=order['type'],
            price=lm.CurrencyAndAmount(
                        amount=0 if pd.isna(order['price']) else order['price'],
                        currency=order['currency']
            ),

            limit_price=lm.CurrencyAndAmount(
                        amount=order['limit_price'],
                        currency=order['limit_currency']
            )            
            if not pd.isna(order['limit_price']) and not pd.isna(order['limit_currency']) else None,
            
            stop_price=lm.CurrencyAndAmount(
                        amount=order['stop_price'],
                        currency=order['stop_currency']
            )
            if not pd.isna(order['stop_price']) and not pd.isna(order['stop_currency']) else None
    )
    
    request=lm.OrderSetRequest(
        order_requests=[request]           
    )

    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],
             o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount,o.price.currency,
             o.limit_price.amount if o.limit_price is not None else "N/A",
             o.limit_price.currency if o.limit_price is not None else "N/A") for o in responses]

pd.DataFrame(attributes, columns=['order_id','client_internal','lusid_instrument_id','side','type',
                                  'state','quantity','price','currency','lim px',
                                  'lim ccy']).style.format({"quantity":"{:20,.0f}","price": "{:20,.2f}"})

### 6.2 Simple PV threshold breach
The orders that we've upserted will fail rule-1 as the order PV will exceed fund's 10% PV rule. 
We can see that with the addition of this new orders, number of instruments that are failing the PV valuation threshold increased from 4 to 5.

In [41]:
order_result=compliance_api.run_compliance(is_pre_trade=True, recipe_id_scope='User', recipe_id_code='valuation_recipe_mid',
                        run_scope=scope,rule_scope=scope)
run_code_order = order_result.run_id.code


In [42]:
order_result = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code_order, rule_scope=scope, rule_code='rule-1')
filtered_breakdowns_orders = [breakdown for breakdown in order_result.rule_result.rule_breakdown if getattr(breakdown, 'group_status', '') == 'Failed']


order_result_df=lusid_response_to_data_frame(filtered_breakdowns_orders)
display(order_result_df[['group_status','results_used.Valuation/PvInReportCcy','results_used.Portfolios.Valuation/PvInReportCcy',
                       'lineage.1.label','lineage.2.sub_label','lineage.5.sub_label','lineage.5.information']])


group_status  results_used.Valuation/PvInReportCcy  \
0       Failed                              772200.0   
1       Failed                              372000.0   
2       Failed                              490500.0   
3       Failed                              488950.0   
4       Failed                            15013500.0   

   results_used.Portfolios.Valuation/PvInReportCcy            lineage.1.label  \
0                                        3419400.0  WithoutExcludedPortfolios   
1                                        3419400.0  WithoutExcludedPortfolios   
2                                        3419400.0  WithoutExcludedPortfolios   
3                                        3419400.0  WithoutExcludedPortfolios   
4                                        3419400.0  WithoutExcludedPortfolios   

           lineage.2.sub_label lineage.5.sub_label    lineage.5.information  
0  simplecompliance2/EQUITY_UK        GB00BH0P3Z91  Instrument/default/Isin  
1  simplecompliance2/EQUITY_UK        GB00BGDT3G23  Instrument/default/Isin  
2  simplecompliance2/EQUITY_UK        GB00BH4HKS39  Instrument/default/Isin  
3  simplecompliance2/EQUITY_UK        GB00B1XZS820  Instrument/default/Isin  
4  simplecompliance2/EQUITY_UK        ES0177542018  Instrument/default/Isin

## 7 Upsert a trade to show its effect
we will now upsert an allocation and run compliance against that. I will first post a small allocation against ORD005 which is a buy of 1 million shares

In [35]:
allocations_df = pd.read_csv('data/iag_allocations.csv')
allocations_df

portfolio instrument_name client_internal          isin          figi  \
0  EQUITY_UK   IAG group plc         EQ_1244  ES0177542018  BBG00J9WBYB9   

   quantity  price currency        allocation_id originating_order  \
0       500   1.48      USD  ALLOC001-for-ORD005            ORD005   

                state side    type  
0  partial allocation  buy  market

In [36]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)
responses = []

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    request = lm.AllocationRequest(
            id=lm.ResourceId(
                scope=scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id = lm.ResourceId(
                scope=scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            side=allocation['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': allocation['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=scope,
                code=portfolio_code
            ),
            state=allocation['state'],
            type=allocation['type'],
            price=lm.CurrencyAndAmount(
                        amount=allocation['price'],
                        currency=allocation['currency']))
    
    request=lm.AllocationSetRequest(
        allocation_requests=[request]           
    )

    response = allocations_api.upsert_allocations(
        allocation_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount) for o in responses]

pd.DataFrame(attributes, columns=['allocation_id','client_internal','lusid_instrument_id','side','type','state','quantity','price']).style.format({"quantity":"{:20,.0f}","price": "${:20,.2f}"})


### 7.2 Simple PV threshold pass 
As this allocation is of quantity 500, it should not breach the 10% PV rule and we expect this to pass rule-1.

In [43]:
alloc_result=compliance_api.run_compliance(is_pre_trade=False, recipe_id_scope='User', recipe_id_code='valuation_recipe_mid',
                        run_scope=scope,rule_scope=scope)
run_code_alloc = alloc_result.run_id.code

In [44]:
alloc_result = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code_alloc, rule_scope=scope, rule_code='rule-1')
filtered_breakdowns_alloc = [breakdown for breakdown in alloc_result.rule_result.rule_breakdown if getattr(breakdown, 'group_status', '') == 'Passed']

# Print the filtered breakdowns
alloc_result_df=lusid_response_to_data_frame(filtered_breakdowns_alloc)
display(alloc_result_df[['group_status','results_used.Valuation/PvInReportCcy','results_used.Portfolios.Valuation/PvInReportCcy',
                       'lineage.1.label','lineage.2.sub_label','lineage.5.sub_label','lineage.5.information']])


group_status  results_used.Valuation/PvInReportCcy  \
0       Passed                              276000.0   
1       Passed                              282000.0   
2       Passed                              308000.0   
3       Passed                               80400.0   
4       Passed                              338400.0   
5       Passed                                2250.0   
6       Passed                                9440.0   

   results_used.Portfolios.Valuation/PvInReportCcy            lineage.1.label  \
0                                        3419400.0  WithoutExcludedPortfolios   
1                                        3419400.0  WithoutExcludedPortfolios   
2                                        3419400.0  WithoutExcludedPortfolios   
3                                        3419400.0  WithoutExcludedPortfolios   
4                                        3419400.0  WithoutExcludedPortfolios   
5                                        3419400.0  WithoutExcludedPortfolios   
6                                        3419400.0  WithoutExcludedPortfolios   

           lineage.2.sub_label lineage.5.sub_label    lineage.5.information  
0  simplecompliance2/EQUITY_UK        GB0002162385  Instrument/default/Isin  
1  simplecompliance2/EQUITY_UK        GB0031348658  Instrument/default/Isin  
2  simplecompliance2/EQUITY_UK        GB0007980591  Instrument/default/Isin  
3  simplecompliance2/EQUITY_UK        GB0005405286  Instrument/default/Isin  
4  simplecompliance2/EQUITY_UK        GB0006043169  Instrument/default/Isin  
5  simplecompliance2/EQUITY_UK        ES0177542018  Instrument/default/Isin  
6  simplecompliance2/EQUITY_UK        GB0008847096  Instrument/default/Isin